In [2]:
# Bibliotecas além do gerenciador Anaconda
!pip install spacy
!python -m spacy download pt_core_news_sm
!pip install wordcloud
!pip install gensim
!pip install tensorflow

2022-05-31 12:08:13.016760: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-31 12:08:13.017931: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


[+] Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [1]:
# Módulos básicos para manuseio de dados e arquivos
import numpy as np
import pandas as pd
import os
from os.path import isfile, join
import nltk
import re
import string
import unicodedata
import json

# Módulos para visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns
#from wordcloud import WordCloud
%matplotlib inline

# Módulo para processamento de linguagem
import spacy

## Carregamento de textos

In [2]:
limited_news_path = r'Software\Fake.br-Corpus' #\fake_10 or \true_10
news_path = r'Software\Fake.br-Corpus\full_texts' #\fake or \true

paths = [limited_news_path, news_path]

In [3]:
def sortDir(dir_path: str, is_meta=False) ->list:
    '''
    Ordena os arquivos dentro de dir_path e os retorna no formato de lista.
    '''
    if is_meta:
        number_separator = "-meta.txt"
    else:
        number_separator = ".txt"

    first_list = os.listdir(dir_path)
    int_list = [int(element.split(number_separator)[0]) for element in first_list]
    int_list.sort()
    final_list = [(str(element) + number_separator) for element in int_list]

    return final_list

def txtToDataframe(path, is_limited=True):
    '''
    Function for converting full texts to a single DataFrame.
    '''
    if is_limited:
        true_files = [path+"\\true_10\\"+f for f in sortDir(dir_path = path+'\\true_10') if isfile(join(path+'\\true_10', f))]
        fake_files = [path+"\\fake_10\\"+f for f in sortDir(dir_path = path+'\\fake_10') if isfile(join(path+'\\fake_10', f))]
    else:
        true_files = [path+"\\true\\"+f for f in sortDir(dir_path = path+'\\true') if isfile(join(path+'\\true', f))]
        fake_files = [path+"\\fake\\"+f for f in sortDir(dir_path = path+'\\fake') if isfile(join(path+'\\fake', f))]
    
    texts = []
    labels = []
    
    for file in true_files:
        with open(file, encoding='utf-8-sig') as f:
            texts.append(f.read())
            labels.append(0)
    for file in fake_files:
        with open(file, encoding='utf-8-sig') as f:
            texts.append(f.read())
            labels.append(1)
            
    df = pd.DataFrame(list(zip(texts,labels)),columns=['texts','labels'])
    
    # Com esta função, textos e labels foram inseridos em um DataFrame de maneira sequencial. Todas as notícias verdadeiras vêm
    # ANTES do bloco de notícias falsas.
    
    return df

def appendMetadata(path,df, is_limited=True):
    '''
    Function for appending metadata to previously generated news DataFrame.
    '''
    if is_limited:
        true_meta = [path+"\\true-meta-information-10\\"+f for f in sortDir(dir_path = path+'\\true-meta-information-10',is_meta=True) if isfile(join(path+'\\true-meta-information-10', f))]
        fake_meta = [path+"\\fake-meta-information-10\\"+f for f in sortDir(dir_path = path+'\\fake-meta-information-10',is_meta=True) if isfile(join(path+'\\fake-meta-information-10', f))]
    else:
        true_meta = [path+"\\true-meta-information\\"+f for f in sortDir(dir_path = path+'\\true-meta-information',is_meta=True) if isfile(join(path+'\\true-meta-information', f))]
        fake_meta = [path+"\\fake-meta-information\\"+f for f in sortDir(dir_path = path+'\\fake-meta-information',is_meta=True) if isfile(join(path+'\\fake-meta-information', f))]
    

    #true_meta e fake_meta são listas com todas os paths para arquivos de metadata.
    
    columns = ["author", "source", "category", "date","tokens","words_without_punctuation","types","number_of_links","uppercase_words","verbs","subjuntive_imperative","nouns","adjectives","adverbs","modal_verbs","singular_first_and_second_personal_pronouns","plural_first_personal_pronouns","pronouns","pausality","characters","avg_sentence_length","avg_word_length","percentage_of_spelling_errors","emotiveness","diversity"]
    
    true_metadata = pd.DataFrame(columns=columns)
    fake_metadata = pd.DataFrame(columns=columns)
    
    for file in true_meta:
        #print(file)
        aux = pd.read_csv(file, header=None, sep = '\n').transpose()
        aux.columns = columns
        true_metadata=true_metadata.append(aux)
        
        
    for file in fake_meta:
        #print(file)
        aux = pd.read_csv(file, header=None, sep = '\n').transpose()
        aux.columns = columns
        fake_metadata=fake_metadata.append(aux)
        
    
    metadata = pd.DataFrame(columns=columns)
    metadata = metadata.append(true_metadata,ignore_index=True)
    metadata = metadata.append(fake_metadata,ignore_index=True) 


    complete_df = pd.concat([df,metadata],axis=1)
    # Este DataFrame possui todos os textos/labels (2 colunas) e metadata (25 colunas).
    
    return complete_df

In [4]:
ai = int(input('''0 - Base com 10 notícias verdadeiras e 10 notícias falsas
1 - Base completa de notícias
'''))

path = paths[ai]

if ai == 0:
    data = txtToDataframe(path) # Dataframe contendo notícias e labels.
    complete_data = appendMetadata(path,data) # Dataframe contendo notícias, labels e metadata.
else:
    data = txtToDataframe(path,is_limited=False)
    complete_data = appendMetadata(path,data,is_limited=False)

0 - Base com 10 notícias verdadeiras e 10 notícias falsas
1 - Base completa de notícias
1


In [5]:
complete_data['texts'][0]

'O Podemos decidiu  expulsar o deputado federal Carlos Gaguim do partido após a Polícia Federal fazer buscas a apreensões no gabinete dele na Câmara. Com isso, a legenda abre espaço para receber a senadora expulsa pelo PMDB, Katia Abreu. Por meio de nota, a legenda informou que o afastamento do parlamentar já era algo acordado entre os filiados da sigla.  “Ainda que o parlamentar tenha comunicado a conclusão de sua desfiliação para esta semana, diante dos fatos noticiados hoje, a Executiva Nacional do Podemos solicita o imediato cancelamento de sua filiação dos quadros do partido”.\n\nO partido, que no passado chegou a cogitar lançar o parlamentar como candidato ao Senado, diz que “apoia a investigação com a ampla apuração dos eventuais crimes cometidos e a consequente responsabilização dos envolvidos, para que todos sejam punidos com o máximo rigor da Lei, independentemente de posição ou cargo ocupado”. '

## Preprocessamento de textos

In [6]:
# Carregando o pacote de língua portuguesa para o processador Spacy
nlp = spacy.load('pt_core_news_sm')

In [7]:
# Defininido funções de preprocessamento

def removePunct(text):
    '''
    Removes any punctuation included in string.punctuation.
    '''
    translator = text.maketrans({key:'' for key in string.punctuation+'“”'}) # Translates any punctuation into ''
    return text.translate(translator)

def removeNumbers(text):
    '''
    Removes any number character in text.
    '''
    return re.sub('[0-9]', '' , text) # Translates any number into ''

def removeStopWords(string):
    '''
    Removes any portuguese stopwords, using Spacy's standard package.
    '''
    doc = nlp(string)
    return ' '.join([token.text for token in doc if token.is_stop is False])

def lemmatize(string):
    '''
    Lemmatizes text word-by-word. Notice that lemmatizing is not as harsh as stemming, which makes the final text easier to read and understand in common language.
    '''
    doc = nlp(string)
    return ' '.join([token.lemma_ for token in doc])

def prep(string, useStopWords = False, lemma = False):
    '''
    Executes previously defined preprocessing in text.
    '''

    result = removeNumbers(removePunct(string)).lower()
    
    if useStopWords and lemma:
        doc = nlp(result)
        result = ' '.join([token.lemma_ for token in doc if token.is_stop is False])
    elif useStopWords:
        doc = nlp(result)
        result = ' '.join([token.text for token in doc if token.is_stop is False])
    elif lemma:
        doc = nlp(result)
        result = ' '.join([token.lemma_ for token in doc])

    result = result.replace('\n',"")
    
    return result

In [8]:
# Realizando preprocessamento de textos presentes no Dataframe de notícias completo.

complete_data['texts'] = complete_data['texts'].apply(prep)


# Assignando variáveis dependentes e independentes

y = complete_data['labels'].values # y is strings for labels; but should be fake-0/true-1
X = [d.split() for d in complete_data['texts'].tolist()] # X is a list of lists of words.

In [9]:
complete_data['texts'][0]

'o podemos decidiu  expulsar o deputado federal carlos gaguim do partido após a polícia federal fazer buscas a apreensões no gabinete dele na câmara com isso a legenda abre espaço para receber a senadora expulsa pelo pmdb katia abreu por meio de nota a legenda informou que o afastamento do parlamentar já era algo acordado entre os filiados da sigla  ainda que o parlamentar tenha comunicado a conclusão de sua desfiliação para esta semana diante dos fatos noticiados hoje a executiva nacional do podemos solicita o imediato cancelamento de sua filiação dos quadros do partidoo partido que no passado chegou a cogitar lançar o parlamentar como candidato ao senado diz que apoia a investigação com a ampla apuração dos eventuais crimes cometidos e a consequente responsabilização dos envolvidos para que todos sejam punidos com o máximo rigor da lei independentemente de posição ou cargo ocupado '

## Tokenization (TensorFlow)

In [10]:
# Importando módulos para tokenização de textos
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [11]:
# Assignando bases de teste e treino

test_limit = 0.10
training_sentences,testing_sentences,training_labels,testing_labels = train_test_split(X,y,test_size=test_limit, random_state=42)

#training_sentences = X[0:test_limit]
#testing_sentences = X[test_limit:]

#training_labels = y[0:test_limit]
#testing_labels = y[test_limit:]

In [12]:
# Análise e truncagem de sequências

def sequenceKill(min_length,max_length,training_sentences_local,training_labels_local,training_sequences_local):

    #plt.hist([len(x) for x in X], bins = 750)
    #plt.show()

    nos = np.array([len(x) for x in training_sentences_local])
    print("There are "+ str(len(nos[nos>=max_length])) + " news equal or longer than "+ str(max_length) + " words (will be truncated).")
    print("There are "+ str(len(nos[nos<min_length])) + " news shorter than "+ str(min_length) + " words (will be killed).")
    
    print("Final database will have "+str(len(training_sentences_local)-len(nos[nos<min_length]))+" total entries.")

    
    flag_dict = {}
    for i in range(0,len(training_sequences_local)):
        if len(training_sequences[i]) < min_length:
            flag_dict[i] = True
        else:
            flag_dict[i] = False

    training_labels_local = [training_labels_local[i] for i in range(0,len(flag_dict)) if (flag_dict[i] == False)]

    training_sequences_local = [seq for seq in training_sequences_local if len(seq)>=min_length]
    
    return training_sequences_local, training_labels_local

In [13]:
# Gerando dicionário de tokens (com base nos textos de treinamento)
tokenizer = Tokenizer()  # não utilizaremos o oov_token='<OOV>'
tokenizer.fit_on_texts(training_sentences)
vocab = tokenizer.word_index

# Realizando a sequencialização das bases de treinamento e teste
training_sequences = tokenizer.texts_to_sequences(training_sentences)
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)

In [14]:
# Analisando a base
# Eliminando menores e truncando maiores

min_length = 150
max_length = 500

training_sequences, training_labels = sequenceKill(min_length, max_length, training_sentences, training_labels,training_sequences)

training_padded = pad_sequences(training_sequences,maxlen=max_length,padding='post',truncating='pre')
#testing_padded = pad_sequences(testing_sequences,maxlen=max_length,padding='post',truncating='pre')

There are 2823 news equal or longer than 500 words (will be truncated).
There are 1586 news shorter than 150 words (will be killed).
Final database will have 4894 total entries.


## Vectorization (gensim)

In [15]:
import gensim

In [16]:
# Gerando o dicionário de vetores para cada palavra da base de treinamento

DIM = 100
w2v_model = gensim.models.Word2Vec(sentences=training_sentences, vector_size=DIM, window=10, min_count=1)

In [33]:
w2v_model.wv.most_similar('faca')

[('briga', 0.8260678052902222),
 ('viatura', 0.7989552617073059),
 ('perna', 0.7813385128974915),
 ('moto', 0.7804338335990906),
 ('facada', 0.7679027318954468),
 ('passeando', 0.7672021985054016),
 ('namorada', 0.7645990252494812),
 ('hemorragia', 0.764021635055542),
 ('parede', 0.7602987289428711),
 ('prece', 0.7588621973991394)]

## Criação da rede neural

In [18]:
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [19]:
# Assignando os vetores gerados para uma matriz de pesos, que será inserida na rede neural 

vocab_size = len(vocab)+1

def get_weight_matrix(model):
    ''' Inserts every word vector in a NumPy array, ordering them by token index (which is determined by their appearing frequencies). '''
    weight_matrix = np.zeros((vocab_size, DIM))

    for word, token in vocab.items():
        weight_matrix[token] = model.wv[word]

    return weight_matrix

embedding_vectors = get_weight_matrix(model=w2v_model)

In [20]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=DIM, weights = [embedding_vectors], input_length=max_length, trainable=False))
model.add(LSTM(units=128))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 100)          9235900   
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 9,353,277
Trainable params: 117,377
Non-trainable params: 9,235,900
_________________________________________________________________


In [21]:
model.fit(training_padded, np.transpose(np.asarray(training_labels)), validation_split=0.3, epochs=10)

Epoch 1/10
108/108 [==============================] - 79s 669ms/step - loss: 0.2990 - acc: 0.8809 - val_loss: 0.2594 - val_acc: 0.9088
Epoch 2/10
108/108 [==============================] - 76s 706ms/step - loss: 0.2773 - acc: 0.9010 - val_loss: 0.2411 - val_acc: 0.9156
Epoch 3/10
108/108 [==============================] - 74s 689ms/step - loss: 0.2386 - acc: 0.9063 - val_loss: 0.2137 - val_acc: 0.9210
Epoch 4/10
108/108 [==============================] - 79s 731ms/step - loss: 0.1890 - acc: 0.9273 - val_loss: 0.2017 - val_acc: 0.9306
Epoch 5/10
108/108 [==============================] - 86s 799ms/step - loss: 0.1750 - acc: 0.9355 - val_loss: 0.1905 - val_acc: 0.9374
Epoch 6/10
108/108 [==============================] - 74s 690ms/step - loss: 0.1498 - acc: 0.9445 - val_loss: 0.1829 - val_acc: 0.9381
Epoch 7/10
108/108 [==============================] - 69s 638ms/step - loss: 0.1307 - acc: 0.9542 - val_loss: 0.2126 - val_acc: 0.9340
Epoch 8/10
108/108 [==============================] - 6

In [22]:
# Sequencialização e truncagem do conjunto de testes

sequences = tokenizer.texts_to_sequences(testing_sentences) # Sequencing testing sentences.
padded = pad_sequences(testing_sequences,maxlen=max_length,padding='post',truncating='pre')


#Tesntando acurácia do modelo na base de testes

y_pred = (model.predict(padded) >=0.5).astype(int)

accuracy_score(testing_labels, y_pred)

0.9597222222222223

## Salvando o modelo

In [22]:
def saveModel(model,dicto):
    name = input('nome do modelo:')
    model.save(str(name)+'.h5')
    with open('wordIndex.json','w') as f:
        json.dump(dicto,f,indent = "")
    print('Model saved!')
    return

saveModel(model, vocab)

KeyboardInterrupt: Interrupted by user

## Buscando notícias

In [ ]:
testing_labels[14]

In [ ]:
testing_sentences[14]